In [22]:
import pickle
import pandas as pd
import re
import numpy as np

In [125]:
# 读取
shanghai_new = pickle.load(open( './shanghai_new.pkl', 'rb'))
hangzhou_new = pickle.load(open( './hangzhou_new.pkl', 'rb'))

In [126]:
l = []
for key in hangzhou_new.keys():
    l.append(len(hangzhou_new[key].keys()))
    
print('爬取房屋的总数：',sum(l))

爬取房屋的总数： 1424


In [127]:
hangzhou_new_df = pd.DataFrame(columns = pd.DataFrame(hangzhou_new['pg1']).index)
for i in hangzhou_new.keys():
    temp = pd.DataFrame(hangzhou_new[i]).T
    hangzhou_new_df = pd.concat([hangzhou_new_df, temp], ignore_index = True)

In [128]:
hangzhou_new_df.head()

,house_name,total_price,unit_price,unit_price_desc,type,location1,area
0,[赞成星谷],[总价100-188(万/套)],[25000],[ 元/㎡(均价)],[商业类],[拱墅],[建面 42-64㎡]
1,[同协金座],[总价157-210(万/套)],[33000],[ 元/㎡(均价)],[商业类],[上城],[建面 45-73㎡]
2,[蓝城久宸里],[总价1850-3600(万/套)],[80000],[ 元/㎡(均价)],[商业类],[上城],[建面 255-515㎡]
3,[云悦湾],[总价170-243(万/套)],[40000],[ 元/㎡(均价)],[商业类],[上城],[建面 41-58㎡]
4,[世茂西溪宸悦],[总价152-588(万/套)],[42000],[ 元/㎡(均价)],[商业类],[西湖],[建面 38-168㎡]


In [129]:
# 发现dataframe中每个元素都是一个列表（有可能为空）
def extract_0(x):
    """
    提取列表中第一个元素，若列表为空，则返回none
    """
    try:
        return x[0]
    except:
        return None   

In [130]:
df_hangzhou = hangzhou_new_df.copy()
# dataframe中每一列数据都从列表中提取出来
for col in df_hangzhou.columns:
    df_hangzhou[col] = df_hangzhou[col].apply(extract_0)

In [131]:
# total_price 需要正则表达式进行处理
# area 需要正则表达式进行处理
# unit_price 全都是’元/㎡(均价)‘
df_hangzhou.head()

,house_name,total_price,unit_price,unit_price_desc,type,location1,area
0,赞成星谷,总价100-188(万/套),25000,元/㎡(均价),商业类,拱墅,建面 42-64㎡
1,同协金座,总价157-210(万/套),33000,元/㎡(均价),商业类,上城,建面 45-73㎡
2,蓝城久宸里,总价1850-3600(万/套),80000,元/㎡(均价),商业类,上城,建面 255-515㎡
3,云悦湾,总价170-243(万/套),40000,元/㎡(均价),商业类,上城,建面 41-58㎡
4,世茂西溪宸悦,总价152-588(万/套),42000,元/㎡(均价),商业类,西湖,建面 38-168㎡


单价

In [132]:
# 发现单价里面有价格待定,将其转换为空值
df_hangzhou['unit_price'] = df_hangzhou.unit_price.replace('价格待定', np.nan)

In [133]:
# 单价里面也有区间，需要进行处理
print('单价为区间的数据个数：',df_hangzhou.unit_price.str.contains('-').sum())

单价为区间的数据个数： 2


In [134]:
# 直接通过链家前端页面可以看到，这两个价格其实是总价
df_hangzhou.loc[df_hangzhou.unit_price.str.contains('-') == True]

,house_name,total_price,unit_price,unit_price_desc,type,location1,area
14,中港西湖院子,None,880-1300,(万/套)(总价),别墅,余杭,建面 150-180㎡
29,悦青蓝,None,200-270,(万/套)(总价),住宅,余杭,建面 87-135㎡


In [135]:
# 对错位数据进行纠正
df_hangzhou.loc[df_hangzhou.unit_price_desc == '\xa0(万/套)(总价)', 'total_price'] = df_hangzhou.loc[df_hangzhou.unit_price_desc == '\xa0(万/套)(总价)', 'unit_price']
df_hangzhou.loc[df_hangzhou.unit_price_desc == '\xa0(万/套)(总价)', 'unit_price'] = np.nan

对总价进行处理

In [136]:
temp = df_hangzhou.total_price.str.extract('总价([\w-]+)')
temp.loc[df_hangzhou.unit_price_desc == '\xa0(万/套)(总价)',0] = df_hangzhou.loc[df_hangzhou.unit_price_desc == '\xa0(万/套)(总价)', 'total_price']

In [137]:
# # 观察发现总价中无法提取数字的，都是没有爬取到总价的（即 总价 为空）
# df_shanghai.loc[temp.isna()[0], 'total_price'].value_counts()

In [138]:
# 获取总价的最大、最小值：
# 有区间的正常取最大、最小值；没有区间的最大最小值都直接取数字
df_hangzhou['total_price_min'] = temp[0].str.extract(r'([\d]+)-')
df_hangzhou['total_price_max'] = temp[0].str.extract(r'-([\d]+)')
df_hangzhou.loc[df_hangzhou['total_price_min'].isna(), 'total_price_min'] = temp.loc[df_hangzhou['total_price_min'].isna(), 0] 
df_hangzhou.loc[df_hangzhou['total_price_max'].isna(), 'total_price_max'] = temp.loc[df_hangzhou['total_price_max'].isna(), 0] 

对面积进行处理

In [140]:
temp = df_hangzhou.area.str.extract(r'([\d-]+)')

In [141]:
# 提取为空都是因为原始的爬取数据就是空的
# df_shanghai.loc[df_shanghai.area.str.extract(r'([\d-]+)').isna()[0],'area'].unique()

In [142]:
df_hangzhou['area_min'] = temp[0].str.extract(r'([\d]+)-')
df_hangzhou['area_max'] = temp[0].str.extract(r'-([\d]+)')
df_hangzhou.loc[df_hangzhou['area_min'].isna(), 'area_min'] = temp.loc[df_hangzhou['area_min'].isna(), 0] 
df_hangzhou.loc[df_hangzhou['area_max'].isna(), 'area_max'] = temp.loc[df_hangzhou['area_max'].isna(), 0] 

单价、总价、面积相关列转换为数值类型

In [145]:
col_list = [ 'unit_price', 'area_min', 'area_max', 'total_price_min', 'total_price_max']

In [146]:
df_hangzhou[col_list] = df_hangzhou[col_list].astype('float')

In [147]:
df_hangzhou.to_csv('hz_new_clean.csv')